COMP721 - Mini Project

216063593 - Benjamin Naidoo

218015230 - Kailin Reddy

# All Imports

In [ ]:
# All Imports

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, explained_variance_score, max_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive/')

# Outlier Detection - KNN

In [2]:
# Defined Methods

def perform_PCA(df):
    x = StandardScaler().fit_transform(df)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data=principalComponents, columns=['principal component 1', 'principal component 2'])
    return principalDf

def outlierDetect(df1, year, start, end, name):
    df = df1.iloc[:, start:end].fillna(0)

    principalDf = perform_PCA(df)
    X = principalDf.values

    nbrs = NearestNeighbors(n_neighbors=3)
    nbrs.fit(X)
    # distances and indexes of k-neighbors from model outputs
    distances, indexes = nbrs.kneighbors(X)  
    # plot mean of k-distances of each observation

    player_playoffs_outlier_index = np.where(distances.mean(axis=1) > 1)

    startIndex = df.index[0]
    updatedOutlierIndices = []
    for i in range(len(player_playoffs_outlier_index[0])):
        updatedOutlierIndices.append(player_playoffs_outlier_index[0][i] + startIndex)

    outlier_values = principalDf.iloc[player_playoffs_outlier_index]
    Y = df1.iloc[player_playoffs_outlier_index]

    data_with_clusters = Y.copy()
    data_with_clusters['PrincipalComponent1'] = 0.0
    data_with_clusters['PrincipalComponent2'] = 0.0

    for i in range(len(updatedOutlierIndices)):
        data_with_clusters['PrincipalComponent1'][updatedOutlierIndices[i]] = principalDf['principal component 1'][player_playoffs_outlier_index[0][i]]
        data_with_clusters['PrincipalComponent2'][updatedOutlierIndices[i]] = principalDf['principal component 2'][player_playoffs_outlier_index[0][i]]

    indexArray = []
    count = 0

    for index in updatedOutlierIndices:
        for out in outlier_values["principal component 1"]:
            if (data_with_clusters['PrincipalComponent1'][index] == out):
                indexArray.append(index)
        count = count + 1

    indexArray2 = []
    count = 0

    for index in updatedOutlierIndices:
        for out in outlier_values["principal component 2"]:
            if (data_with_clusters['PrincipalComponent2'][index] == out):
                indexArray2.append(index)
        count = count + 1

    finalIndexArray = []

    for m in indexArray:
        for n in indexArray2:
            if (m == n):
                finalIndexArray.append(m)

    if (year < 0):
        print('The Outstanding players for ' + name + ' are : ')
    else:
        print('The Outstanding players for ' + name + ' in the year ' + str(year) + ' are : ')
    print('================================================================================')
    for i in finalIndexArray:
        print(data_with_clusters['firstname'][i] + ' ' + data_with_clusters['lastname'][i])

    print("===============================================================================")

    # plot outlier values
    plt.scatter(principalDf["principal component 1"], principalDf["principal component 2"], color="gray",)
    plt.scatter(outlier_values["principal component 1"], outlier_values["principal component 2"], color="red")
    plt.show()

def readPlayerFiles(filename, start, end, name):
    df = pd.read_csv(filename, header=0).fillna(0)


    if "year" in df.columns:
        df1 = df.sort_values(by='year')
        df1 = df1.reset_index(drop=True)

        dataFrameArr = []
        years = []
        for year in df1["year"]:
            if (year not in years):
                years.append(year)

        for year in years:
            newDataFrame = df1.query("year==" + str(year))
            dataFrameArr.append(newDataFrame)

        for i in range(len(dataFrameArr)):
            newOutlier = dataFrameArr[i]
            outlierDetect(newOutlier, years[i], start, end, name)
            print()
            print()

    else:
        outlierDetect(df, -1, start, end, name)
        print()
        print()

# Outlier Detection - KMeans

In [4]:
# Defined Methods

def perform_PCA(df):
    x = StandardScaler().fit_transform(df)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data=principalComponents, columns=['principal component 1', 'principal component 2'])
    return principalDf


def get_Dist(x1, y1, x2, y2):
    diffX = x2 - x1
    diffY = y2 - y1
    if diffX <= 0 and diffY <= 0:
        return 0
    else:
        return math.sqrt(math.pow(x2 - x1, 2) + math.pow(y2 - y1, 2))


def outlierDetect(df, year, start, end, name):
    df1 = df.iloc[:, start:end]

    principalDF = perform_PCA(df1)

    model = KMeans(n_clusters=1, random_state=0)
    identified_clusters = model.fit(principalDF[['principal component 1', 'principal component 2']])
    clusterCenter = model.cluster_centers_[0]
    x1 = clusterCenter[0]
    y1 = clusterCenter[1]

    identified_clusters1 = identified_clusters.predict(principalDF[['principal component 1', 'principal component 2']])

    data_with_clusters = df.copy()

    data_with_clusters['Clusters'] = identified_clusters1
    data_with_clusters['PC1'] = 0.0
    data_with_clusters['PC2'] = 0.0
    startIndex = data_with_clusters.index[0]

    for i in range(len(data_with_clusters['firstname'])):
        data_with_clusters['PC1'][i + startIndex] = principalDF['principal component 1'][i]
        data_with_clusters['PC2'][i + startIndex] = principalDF['principal component 2'][i]

    data_with_clusters['distFromCentre'] = 0.0

    for i in range(len(data_with_clusters['PC1'])):
        data_with_clusters['distFromCentre'][i + startIndex] = get_Dist(x1, y1, data_with_clusters["PC1"][i + startIndex], data_with_clusters['PC2'][i + startIndex])

    thresh1 = (max(data_with_clusters["PC1"]) + max(data_with_clusters["PC2"])) / 2
    df = data_with_clusters.query("distFromCentre > " + str(thresh1))

    if (year < 0):
        print('The Outstanding players for ' + name + ' are : ')
    else:
        print('The Outstanding players for ' + name + ' in the year ' + str(year) + ' are : ')

    print('================================================================================')

    for i in df.index:
        print(df['firstname'][i] + ' ' + df['lastname'][i])

    print("================================================================================")

    plt.scatter(principalDF['principal component 1'], principalDF['principal component 2'], color='gray')
    plt.scatter(df['PC1'], df['PC2'], color='red')
    plt.show()

def readPlayerFiles(filename, start, end, name):
    df = pd.read_csv(filename, header=0).fillna(0)

    if "year" in df.columns:
        df1 = df.sort_values(by='year')
        df1 = df1.reset_index(drop=True)

        dataFrameArr = []
        years = []
        for year in df1["year"]:
            if (year not in years):
                years.append(year)

        for year in years:
            newDataFrame = df1.query("year==" + str(year))
            dataFrameArr.append(newDataFrame)

        for i in range(len(dataFrameArr)):
            newOutlier = dataFrameArr[i]
            outlierDetect(newOutlier, years[i], start, end, name)
            print()
            print()


    else:
        outlierDetect(df, -1, start, end, name)
        print()
        print()

# Running Outlier Detection

In [ ]:
# Running code
# Run either KNN or KMeans block to define methods before running this block

readPlayerFiles("/content/drive/MyDrive/Colab Notebooks/Data/player_regular_season_career.txt", 8, 23, "NBA Regular Seasons in general")
print("===================================================================")

readPlayerFiles("/content/drive/MyDrive/Colab Notebooks/Data/player_playoffs_career.txt", 8, 21, "NBA Playoffs in General")
print("===================================================================")

readPlayerFiles("/content/drive/MyDrive/Colab Notebooks/Data/player_regular_season.txt", 6, 23, "NBA Regular Seasons")
print("===================================================================")

readPlayerFiles("/content/drive/MyDrive/Colab Notebooks/Data/player_playoffs.txt", 8, 23, "NBA Playoffs")
print("===================================================================")

readPlayerFiles("/content/drive/MyDrive/Colab Notebooks/Data/player_allstar.txt", 8, 23, "NBA All Star")

# Game Outcome Prediction

In [12]:
# Defined Methods
def get_win_rate(teamSeasonRow):
		return teamSeasonRow["won"] / (teamSeasonRow["won"] + teamSeasonRow["lost"])
  
def get_o_3pm_value(teamSeasonRow):
		teamSeasonWinRate = get_win_rate(teamSeasonRow)
		if teamSeasonRow["year"] < 1979:
			o_3pmMultiplier = 1 + ((teamSeasonWinRate - averageWinRate) / averageWinRate)
			return int(round(o_3pmAverage * o_3pmMultiplier, ndigits=0))
		else:
			return teamSeasonRow["o_3pm"]

def get_d_3pm_value(teamSeasonRow):
		teamSeasonWinRate = get_win_rate(teamSeasonRow)
		if teamSeasonRow["year"] < 1979:
			d_3pmMultiplier = 1 + ((teamSeasonWinRate - averageWinRate) / averageWinRate)
			return int(round(d_3pmAverage * d_3pmMultiplier, ndigits=0))
		else:
			return teamSeasonRow["d_3pm"]

def get_o_3pa_value(teamSeasonRow):
		if teamSeasonRow["year"] < 1999:
			return int(round(teamSeasonRow["o_3pm"] * o_3paMultiplier, ndigits=0))
		else:
			return teamSeasonRow["o_3pa"]

def get_d_3pa_value(teamSeasonRow):
		if teamSeasonRow["year"] < 1999:
			return int(round(teamSeasonRow["d_3pm"] * d_3paMultiplier, ndigits=0))
		else:
			return teamSeasonRow["d_3pa"]

In [ ]:
# Load Data
teamSeasonDF = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/team_season.txt", header=0)
teamSeasonDF.drop(teamSeasonDF[teamSeasonDF.year < 1976].index, inplace=True)
teamSeasonDF.reset_index(inplace=True, drop=True) 

# Model Training
labelsDF = pd.DataFrame(columns=["win_rate"])
labelsDF["win_rate"] = teamSeasonDF.apply(get_win_rate, axis=1)

averageWinRate = teamSeasonDF["won"].sum() / (teamSeasonDF["won"].sum() + teamSeasonDF["lost"].sum())

teamSeason1979DF = teamSeasonDF[teamSeasonDF.year >= 1979]

o_3pmDF = teamSeason1979DF["o_3pm"]
o_3pmAverage = o_3pmDF.sum() / len(teamSeason1979DF.index)

d_3pmDF = teamSeason1979DF["d_3pm"]
d_3pmAverage = d_3pmDF.sum() / len(teamSeason1979DF.index)

teamSeasonDF["o_3pm"] = teamSeasonDF.apply(get_o_3pm_value, axis=1)
teamSeasonDF["d_3pm"] = teamSeasonDF.apply(get_d_3pm_value, axis=1)

teamSeason1999DF = teamSeasonDF[teamSeasonDF.year >= 1999]

o_3pmDF = teamSeason1999DF["o_3pm"]
o_3paDF = teamSeason1999DF["o_3pa"]
o_3paMultiplier = o_3paDF.sum() / o_3pmDF.sum()

d_3pmDF = teamSeason1999DF["d_3pm"]
d_3paDF = teamSeason1999DF["d_3pa"]
d_3paMultiplier = d_3paDF.sum() / d_3pmDF.sum()

teamSeasonDF["o_3pa"] = teamSeasonDF.apply(get_o_3pa_value, axis=1)
teamSeasonDF["d_3pa"] = teamSeasonDF.apply(get_d_3pa_value, axis=1)

# Create dataframe with only required features
featureMatrixDF = teamSeasonDF.drop(["team", "year", "leag", "won", "lost"], axis=1)

featuresTrain, featuresTest, labelsTrain, labelsTest = train_test_split(featureMatrixDF, labelsDF, train_size=0.80, random_state=1)

scaler = preprocessing.StandardScaler()
scaler.fit(featuresTrain)

featuresTrainNormalised = pd.DataFrame(scaler.transform(featuresTrain), columns=featuresTrain.columns)
featuresTestNormalised = pd.DataFrame(scaler.transform(featuresTest), columns=featuresTest.columns)

mlpRegressor = MLPRegressor(hidden_layer_sizes=(100, 100, 100), solver="lbfgs", activation="tanh", random_state=1, max_iter=10000)
svRegressor = SVR()

for algorithm in [mlpRegressor, svRegressor]:
		print("Algorithm:", algorithm.__class__.__name__)
		print()
		algorithm.fit(featuresTrainNormalised, np.ravel(labelsTrain))
		labelPredictions = algorithm.predict(featuresTestNormalised)
		print("\tPrediction Metrics:")
		print("\t\tMax Error: ", round(max_error(labelsTest, labelPredictions), 3))
		print("\t\tExplained Variance Score: ", round(explained_variance_score(labelsTest, labelPredictions), 3))
		print("\t\tMean Absolute Error: ", round(mean_absolute_error(labelsTest, labelPredictions), 3))
		print("\t\tMean Squared Error: ", round(mean_squared_error(labelsTest, labelPredictions), 3))
		print("\t\tMedian Absolute Error: ", round(median_absolute_error(labelsTest, labelPredictions), 3))
		print("\t\tR2 Score: ", round(r2_score(labelsTest, labelPredictions), 3))
		print()

In [ ]:
# Load teams.txt
teamsFile = "/content/drive/MyDrive/Colab Notebooks/Data/teams.txt"
teamsDF = pd.read_csv(teamsFile)
print(teamsDF.head())

In [ ]:
# Choose 2 teams for comparison
compareDF = teamSeasonDF.sample(n=2)
print(compareDF)

In [ ]:
# Display teams for comparison
team1Label = compareDF.iloc[0, 0]
team1year = compareDF.iloc[0, 1]

team2Label = compareDF.iloc[1, 0]
team2year = compareDF.iloc[1, 1]

team1row = teamsDF.loc[teamsDF['team']==team1Label]
team1 = "Team 1 : " + str(team1row.iloc[0, 1]) + " " + str(team1row.iloc[0, 2])
print(team1)
print("Year : ", team1year)
print()

team2row = teamsDF.loc[teamsDF['team']==team2Label]
team2 = "Team 2 : " + str(team2row.iloc[0, 1]) + " " + str(team2row.iloc[0, 2])
print(team2)
print("Year : ", team2year)

compareDF = compareDF.drop(['won', 'lost', 'year', 'leag', 'team'], axis=1)

In [ ]:
# Prediction
print("Model - MLP Regression")
print()

winProbability = mlpRegressor.predict(compareDF)
winProbT1 = winProbability[0]
winProbT2 = winProbability[1]

print("Win Probability of Team 1: ", winProbT1)
print("Win Probability of Team 2: ", winProbT2)
print()

# Normalization
normWinProbT1 = winProbT1 / (winProbT1 + winProbT2)
normWinProbT2 = winProbT2 / (winProbT1 + winProbT2)

print("The probability of Team 1 beating Team 2 is: ", normWinProbT1)
print("The probability of Team 2 beating Team 1 is: ", normWinProbT2)

if (normWinProbT1 > normWinProbT2) :
  winner = team1
else :
  winner = team2

print()
print("The winner is ", winner)